### Import packages and prepare variables
We are using a chessboard of shape 10x7 for calibration 
You need to adjust the *pattern_size* variable to your size. 

In [1]:
import cv2
import numpy as np
pattern_size = (7,10)
samples_left= []
samples_right= []
left_points = []
right_points = []
img_size = None

### Start stereo stream
We start stereo stream 
1. Switch of Autofocus to prevent the images going in and out of focus
2. Switch of Autowhite balance to prevent change in temprature gradient of image

You might need to remove these features based on your camera.
To check if your camera has these properties you can run the following code in terminal

    sudo apt install v4l-utils 
    v4l2-ctl --list-devices

If you find your camera here find its port number (*mostly of form /dev/videoX*) for example in my case my camera was listed as
    
    C922 Pro Stream Webcam (usb-0000:00:14.0-1):
	/dev/video1

So next we use the code

    v4l2-ctl -d /dev/video1 --list-ctrls
    
This will list all the options and see if it has the options 
1. focus_auto
2. white_balance_temperature_auto

If *focus_auto* is absent remove the line 

    cap1.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    
If *white_balance_temperature_auto* is absent remove the line 

    cap1.set(cv2.CAP_PROP_AUTO_WB, 0)

In [2]:
cap_left = cv2.VideoCapture(1)
cap_left.set(cv2.CAP_PROP_AUTOFOCUS, 1)
cap_left.set(cv2.CAP_PROP_AUTO_WB, 2)
cap_right = cv2.VideoCapture(2)
cap_right.set(cv2.CAP_PROP_AUTOFOCUS, 1)
cap_right.set(cv2.CAP_PROP_AUTO_WB, 2)

True

### Mark chessboard
We perform the following steps to get checkerboard images for calibration
1. Capture frame from both the camera stream simultaneously
2. Find Chessboard Patterns on them
3. draw Chessbaord Corners on the frame and take out coarse corners from the image
4. Caputre and store the images if the chessboards are taken correctly
> Try capturing atleast 50 images from all angles for more precise distortion values

In [3]:
while True:
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()
    if not ret_left:
        break
    if not ret_right:
        break
    if img_size is None:
        img_size = (frame_left.shape[1], frame_right.shape[0])
    
    res_left, corners_left = cv2.findChessboardCorners(frame_left, pattern_size)
    res_right, corners_right = cv2.findChessboardCorners(frame_right, pattern_size)
    
    img_show_left = np.copy(frame_left)
    img_show_right = np.copy(frame_right)
    cv2.drawChessboardCorners(img_show_left, pattern_size, corners_left, res_left)
    cv2.drawChessboardCorners(img_show_right, pattern_size, corners_right, res_right)
    cv2.putText(img_show_left, 'Left Samples captured: %d' % len(samples_left), (0, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    cv2.putText(img_show_right, 'Right Samples captured: %d' % len(samples_right), (0, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    both = np.concatenate((img_show_left, img_show_right), axis=1)
    cv2.imshow('chessboard', both)
    
    wait_time = 0 if (res_left and res_right) else 30
    k = cv2.waitKey(wait_time)
    
    if k == ord('s') and res_left and res_right:
        samples_left.append((cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY), corners_left))
        samples_right.append((cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY), corners_right))
    elif k == 27:
        break

cap_left.release()
cap_right.release()
cv2.destroyAllWindows()

### Correct and Optimize chessboard corner position
We use the criteria of minimal error and maximum iterations which can be altered as you want.

Fucntion used it [cv2.cornerSubPix](https://docs.opencv.org/3.0-beta/modules/imgproc/doc/feature_detection.html) for corner correction based on old values. It is an itervative algorithm for finding correct location of corner pixel using image gradient.

In [4]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 1e-3)

for i in range(len(samples_left)):
    img_left, corners_left = samples_left[i]
    corners_left = cv2.cornerSubPix(img_left, corners_left, (10, 10), (-1,-1), criteria)
    left_points.append(corners_left)
for j in range(len(samples_right)):
    img_right, corners_right = samples_right[j]
    corners_right = cv2.cornerSubPix(img_right, corners_right, (10, 10), (-1,-1), criteria)
    right_points.append(corners_right)

### Initialize Pattern points set for correction

In [5]:
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points = [pattern_points] * len(left_points)

### Finally find the stereo parameters

In [6]:
err, Kl, Dl, Kr, Dr, R, T, E, F = cv2.stereoCalibrate(
    pattern_points, left_points, right_points, None, None, None, None, img_size, flags=0)

### Save the stereo rig paramters in stereo.npy file which can be loaded now as and when required
1. Kl : Camera matrix left camera
2. Dl : Distortion matrix left camera
3. Kr : Camera matrix right camera
4. Dr : Distortion matrix right camera
5. T : Translation matrix
6. R : Rotation Matrix
7. E : Essesntial Matrix
8. F : Fundamental Matrix

In [7]:
np.save('stereo.npy', {'Kl': Kl, 'Dl': Dl, 'Kr': Kr, 'Dr': Dr, 'R': R, 'T': T, 'E': E, 'F': F, 
                       'img_size': img_size, 'left_pts': left_points, 'right_pts': right_points})

In [ ]:
xd = np.load('stereo.npy',allow_pickle=True).item()